In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dogan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dogan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [38]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [39]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

376 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [36]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [37]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
329/329 [==============================] - 0s 1ms/step - loss: 1.9963 - accuracy: 0.2857
Epoch 2/200
329/329 [==============================] - 0s 524us/step - loss: 1.2110 - accuracy: 0.6444
Epoch 3/200
329/329 [==============================] - 0s 373us/step - loss: 0.6553 - accuracy: 0.8267
Epoch 4/200
329/329 [==============================] - 0s 399us/step - loss: 0.4071 - accuracy: 0.8632
Epoch 5/200
329/329 [==============================] - 0s 346us/step - loss: 0.2456 - accuracy: 0.9514
Epoch 6/200
329/329 [==============================] - 0s 352us/step - loss: 0.1791 - accuracy: 0.9696
Epoch 7/200
329/329 [==============================] - 0s 445us/step - loss: 0.1259 - accuracy: 0.9696
Epoch 8/200
329/329 [==============================] - 0s 364us/step - loss: 0.1090 - accuracy: 0.9787
Epoch 9/200
329/329 [==============================] - 0s 391us/step - loss: 0.0897 - accuracy: 0.9848
Epoch 10/200
329/329 [==============================] - 0s 455us/step - los

329/329 [==============================] - 0s 418us/step - loss: 0.0073 - accuracy: 0.9970
Epoch 80/200
329/329 [==============================] - 0s 309us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 81/200
329/329 [==============================] - 0s 312us/step - loss: 0.0125 - accuracy: 0.9970
Epoch 82/200
329/329 [==============================] - 0s 358us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 83/200
329/329 [==============================] - 0s 315us/step - loss: 0.0078 - accuracy: 0.9970
Epoch 84/200
329/329 [==============================] - 0s 306us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 85/200
329/329 [==============================] - 0s 306us/step - loss: 0.0123 - accuracy: 0.9939
Epoch 86/200
329/329 [==============================] - 0s 357us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 87/200
329/329 [==============================] - 0s 340us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 88/200
329/329 [==============================] - 0s 411us/step - loss:

329/329 [==============================] - 0s 394us/step - loss: 9.8470e-04 - accuracy: 1.0000
Epoch 157/200
329/329 [==============================] - 0s 427us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 158/200
329/329 [==============================] - 0s 336us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 159/200
329/329 [==============================] - 0s 349us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 160/200
329/329 [==============================] - 0s 310us/step - loss: 0.0035 - accuracy: 1.0000
Epoch 161/200
329/329 [==============================] - 0s 338us/step - loss: 8.4561e-04 - accuracy: 1.0000
Epoch 162/200
329/329 [==============================] - 0s 352us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 163/200
329/329 [==============================] - 0s 306us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 164/200
329/329 [==============================] - 0s 379us/step - loss: 0.0055 - accuracy: 0.9970
Epoch 165/200
329/329 [==============================] - 0s 3